In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [3]:
X = iris.data
y = iris.target

In [4]:
X.shape, y.shape

((150, 4), (150,))

In [5]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

###  다중분류에 경우는 출력 뉴런의 개수가 정답데이터랑 같아야 한다.

In [6]:
y_one_hot = pd.get_dummies(y)

y_one_hot

,0,1,2
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


- 1. 신경망 구조설계
- 2. 학습/평가 방법 설정
- 3. 학습 및 평가

In [7]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [8]:
# 1. 신경망 구조 설계
# 출력층의 뉴런 개수 / 출력층의 활성화 함수 종류

### 신경망 구조 설계

In [9]:
#1. 신경망 구조 설계
model = Sequential()

# 입력층(input_dim) + 중간층(Dense)
# 첫 입력층(데이터의 특성 개수) input_dim(17) , 10(다음 층 뉴런 10개)
# activation : 활성화함수를 설정(들어온 자극(데이터)에 대한 응답여부를 결정하는 함수)

model.add(Dense(10, input_dim=4, activation="sigmoid"))

# 중간층
model.add(Dense(16, activation="sigmoid")) # 하나의 층

# 출력층
# 출력층은 회귀(활성화함수x), 2진분류(sigmoid), 다중분류에 따라서 사용하는 함수가 달라짐
# 출력층 뉴런개수 : 정답데이터를 원핫인코딩한 데이터 개수
model.add(Dense(3, activation="softmax")) 

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                50        
_________________________________________________________________
dense_1 (Dense)              (None, 16)                176       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 51        
Total params: 277
Trainable params: 277
Non-trainable params: 0
_________________________________________________________________


In [10]:
# 2. 학습/평가 방법 설정
# loss값 categorical_crossentropy로 설정

### 신경망 학습/평가 방법 설정

In [11]:
# 2. 학습 / 평가 방법 설정
# binary_crossentropy : 2진분류에 사용하는 손실함수(비용함수)
# categorical_crossentropy : 다중분류에 사용하는 손실함수
# -> 오차의 평균을 구하는것은 mse와 같지만 0~1사이 값으로 변환 후 평균오차를 구함
# 
model.compile(loss="categorical_crossentropy",  
              optimizer = "Adam",   # 최적화 함수 : 확률적경사하강법 사용  // SGD도 이외에도 Adam도 있음 
              metrics=["acc"]       # metrics : 평가방법을 설정 (정확도 체크)
                                    # 분류 문제이기 때문에 정확도를 확인
             )

# 회귀는 오차만 확인하면 되지만 분류는 정확도까지 확인해줘야 함!!!!!

###  학습 및 학습과정 

In [12]:
model.fit(X,y_one_hot, epochs=100) # epochs : 학습 횟수를 설정

Train on 150 samples
Epoch 1/100
 32/150 [=====>........................] - ETA: 2s

InternalError:  Blas GEMM launch failed : a.shape=(32, 4), b.shape=(4, 10), m=32, n=10, k=4
	 [[node sequential/dense/MatMul (defined at <ipython-input-12-cd1a995a4294>:1) ]] [Op:__inference_distributed_function_731]

Function call stack:
distributed_function


### 모델평가 

In [ ]:
model.evaluate(X,y_one_hot)

### 1. 회귀
- loss = mse(평균제곱오차)
- 출력층 뉴런 개수 : 1
- 출력층의 활성화 함수 : linear(항등함수) -> 디폴트 값
### 2. 2진분류
- loss = binary_crossentropy
- 출력층 뉴런 개수 : 1
- 출력층의 활성화 함수 : sigmoid
### 3. 다중분류
- loss = categorical_crossentropy
- 출력층 뉴런 개수 : 원핫인코딩 된 레이블의 개수
- 출력층의 비활성화 함수 : softmax